In [2]:
import scipy
import numpy as np
from os import listdir
from skimage import data
from PIL import Image as PImage
import matplotlib.pyplot as plt
from scipy.spatial import distance
from skimage.color import label2rgb
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
import pprint
from os import listdir
from matplotlib import image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import random

In [147]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])


def cut_and_gray(img,frame):
    if (frame[0] == "nan") or (frame[1] == "nan") or (frame[2] == "nan") or (frame[3] == "nan") :
        return -2
    else :
        x = int(float(frame[0]))
        y = int(float(frame[1]))
        width = int(float(frame[2]))
        height = int(float(frame[3]))
        imgs=[]        
        w=width//4
        h=height//2
        if w == 0:
            w=1
        if h == 0:
            h = 1
        for j in range (0,2):
            for i in range (0,4):
                imgs.append(rgb2gray(img[(y+(j*h)):(y+((j+1)*h)),(x+(i)*w):(x+(i+1)*w)]))
        return imgs #bounding box's eight parts
    
    
def hist_generator(dic):
    histogram = {}
    for frame in range(1,len(dic)+1):
        if dic[frame] != -2:
            for val in range(len(dic[frame])):
                eight=[]
                for part in range(8):
                    bins=[]
                    for i in range(0,256):
                        bins.append(0)
                    for i in range(0, 256):
                        x = dic[frame][val][part]
                        for j in x:
                            if (i==j):
                                bins[i] += 1/len(x)   
                    eight.append(bins)
                if val != 0:
                    histogram[frame].append(eight)
                if val == 0:
                    histogram[frame] = [eight]
        else:
            histogram[frame] = [-2]
        if frame % 100 == 0:
            print("{} masks done.".format(frame))
    return histogram


def hist_generator_for_gt(dic):
    histogram = {}
    for frame in dic:
        if dic[frame] != -2:
            eight=[]
            for part in range(8):
                bins=[]
                for i in range(0,256):
                    bins.append(0)
                for i in range(0, 256):
                    for j in dic[frame][part]:
                        if (i==j):
                            bins[i] += 1/len(dic[frame][part])   
                eight.append(bins)
            if frame in histogram:
                histogram[frame].append(eight)
            else:
                histogram[frame] = [eight]
        else:
            histogram[frame] = [-2]
            print("{} gt done.".format(frame))
    return histogram


def concat_lbp(img, method1):
    concats = {}
    for i in range(len(img)):
        concat = []
        if img[i] != -2:
            for j in range(0,8):
                concat.append(local_binary_pattern(img[i][j], n_points, radius, method1))    
            concats[i+1] = concat
        else:
            concats[i+1] = -2
    return concats


def stack(dic):
    stacked = {}
    for frame in range(len(dic)):
        if dic[frame] != -2:
            for val in dic[frame]:
                eight=[]
                for part in range(8):
                    eight.append(np.hstack(val[part]))
                if frame in stacked:
                    stacked[frame].append(eight)
                else:
                    stacked[frame] = [eight]
        else:
            stacked[frame] = -2
    return stacked


def stack_for_gt(dic):
    stacked = {}
    for frame in dic:
        if dic[frame] != -2:
            eight=[]
            for part in range(8):
                eight.append(np.hstack(dic[frame][part]))
            if frame in stacked:
                stacked[frame].append(eight)
            else:
                stacked[frame] = [eight]
        else:
            stacked[frame] = -2
    return stacked


def chi_for_hist (mask,gt):
    hist_sum = 0
    for part in range(8):
        for i in range(len(mask[part])):
            hist_sum += 0.5*np.sum((mask[part][i]-gt[part][i])**2/(mask[part][i]+gt[part][i]+1e-6))
    return hist_sum


def dist_collector(his,mask):
    hist_distances = {}
    for i in range (1,(order+1)):
        for j in range(len(mask[i])):
            if his[i] != [-2] and mask[i][j] != -2:
                try:
                    hist_distances[i].append(chi_for_hist(mask[i][j], his[i][0]))
                except KeyError:
                    hist_distances[i] = [chi_for_hist(mask[i][j], his[i][0])]
                    
            elif his[i] == [-2] and mask[i][j] != -2:
                hist_distances[i] = [-2, -1]
            elif his[i] != [-2] and mask[i][j] == -2:
                hist_distances[i] = [-1, -2]
            else:
                hist_distances[i] = [-2, -2]
    return hist_distances


def dist_collector2(his):
    hist_distances = {}
    hist_distances[1] = 0
    for i in range(2,len(his)):
        if his[i-1] != [-2] and his[i] != [-2]:
            hist_distances [i] = chi_for_hist(his[i-1][0],his[i][0])
        elif his[i-1] == [-2] and his[i] != [-2]:
            hist_distances[i] = 0
        elif his[i-1] == [-2] and his[i] == [-2]:
            his_distances[i] = -2
        elif his[i-1] != [-2] and his[i] == [-2]:
            hist_distances[i] = -2
    return hist_distances


METHOD = ['var','default',"uniform","ror"]
radius = 3
n_points = 8

In [4]:
loaded_images = {}
for filename in listdir('images/{}'.format("ballet")):
    # load image
    img_data = image.imread('images/{}/'.format("ballet") + filename)
    # store loaded image
    order=int(filename[0:8]) # Dizi sırasını normalize ettik.
    loaded_images[order]=img_data

In [5]:
frames_gt = []
for line in open('VOT2019_LT/gt/{}_groundtruth.txt'.format("ballet"),"r").readlines():
    X = line.strip()
    frames_gt.append(X.split(","))

In [6]:
grey_eight_imgs_gt = {}
for i in range(len(frames_gt)):
    grey_eight_imgs_gt[i]= cut_and_gray(loaded_images[i+1],frames_gt[i])

In [7]:
boat_masks = {}
frame_number = {}
for line in open('VOT2019_LT/Mask/{}_mask'.format("ballet"),"r").readlines():
    M = line.strip()
    splitted = M.split("\t")
    try:
        boat_masks[int(splitted[0])].append(splitted[1:5])
    except KeyError:
        boat_masks[int(splitted[0])] = [splitted[1:5]]
    
    if (splitted[0]) in frame_number:
        frame_number[int(splitted[0])] += 1
    else:
        frame_number[int(splitted[0])] = 1

for i in range (order):
    if i not in frame_number:
        frame_number[i] = 0

In [8]:
len(grey_eight_imgs_gt[0])

8

In [138]:
grey_mask = {}
#order is a number that how many image i have
for mask in range(1,(order+1)):
    if mask in boat_masks:
        for val in range(len(boat_masks[mask])):
            concat = []
            eight_pics = cut_and_gray(loaded_images[mask], boat_masks[mask][val])
            for a in range(0,8):
                concat.append(local_binary_pattern(eight_pics[a], n_points, radius, "default"))
                
            if mask in grey_mask:
                grey_mask[mask].append(concat)
            else:
                grey_mask[mask] = [concat]
    else:
        grey_mask[mask] = -2

In [140]:
stack_mask = stack(grey_mask)

In [144]:
mask_hists = hist_generator(stack_mask)

In [13]:
concat_gt_d = concat_lbp(grey_eight_imgs_gt, 'default')

In [123]:
len(concat_gt_d[1])

8

In [14]:
stack_gt = stack_for_gt(concat_gt_d)

In [17]:
gt_hists = hist_generator(stack_gt)

In [41]:
distance_gt_gt = dist_collector2(gt_hists)

In [50]:
distance_gt_gt

{1: 0,
 2: 0.5641071581159887,
 3: -2,
 4: 0,
 5: 0.671112790400514,
 6: 0.6847036890823589,
 7: 0.6080429516476945,
 8: 0.6352288811672309,
 9: 0.5388916397896383}

In [146]:
distance_gt_mask

{1: [2.2519371397211843,
  4.389108737304519,
  4.751353365059159,
  4.459703953696372,
  4.593577090414403,
  3.5541442104219314,
  4.3904900969070715],
 2: [2.8670263893336703,
  4.329169013742962,
  3.9817802906038318,
  4.424842991947909,
  4.66029444296091,
  4.659984296976917,
  3.2683439732280353],
 3: [-2, -1],
 4: [-1, -2],
 5: [2.3966167501447533,
  3.1217674757309126,
  4.416175042313419,
  4.415732001871033,
  3.0850782142320297,
  4.670788887396813,
  4.488317373283827,
  4.03511555099926],
 6: [2.4480176668948443,
  4.3637424944231835,
  4.333229630047997,
  3.2744517808639046,
  4.640660490342133,
  4.742655213884014],
 7: [2.4933944555916945,
  3.0076255318706817,
  4.691573760278574,
  3.0047476461335925,
  4.50160641048062,
  4.265027238921658,
  4.282331658392055,
  4.394495696593103],
 8: [-1, -2],
 9: [-1, -2],
 10: [2.4603692696418658,
  3.731335943066003,
  4.361167243736728,
  4.793662548062658,
  4.340802938446892]}

In [111]:
for i in range(1,len(mask_hists)):
    if mask_hists[i] == [-2]:
        print(i)

4
8
9


In [41]:
for i in gt_hists:
    if i == [-2]:
        pprint.pprint(i)

In [145]:
distance_gt_mask = dist_collector(gt_hists, mask_hists)

In [55]:
mask_hists[1][0] == mask_hists[1][6]

True

In [32]:
distance_gt_mask

{1: [2.2519371397211843,
  2.2519371397211843,
  2.2519371397211843,
  2.2519371397211843,
  2.2519371397211843,
  2.2519371397211843,
  2.2519371397211843],
 2: [2.8670263893336703,
  2.8670263893336703,
  2.8670263893336703,
  2.8670263893336703,
  2.8670263893336703,
  2.8670263893336703,
  2.8670263893336703],
 3: [-2, -1],
 4: [-1, -2],
 5: [2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533,
  2.3966167501447533],
 6: [2.4480176668948443,
  2.4480176668948443,
  2.4480176668948443,
  2.4480176668948443,
  2.4480176668948443,
  2.4480176668948443],
 7: [2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945,
  2.4933944555916945],
 8: [-1, -2],
 9: [-1, -2],
 10: [2.4603692696418658,
  2.4603692696418658,
  2.4603692696418658,
  2.4603692696418658,
  2.4603692696418658]}

In [ ]:
# load all images in a directory
for videos in listdir('images/'):
    if videos in ["bicycle"]:
        pprint.pprint("Bicycle video frame started!")
        loaded_images = {}
        for filename in listdir('images/{}'.format(videos)):
            # load image
            img_data = image.imread('images/{}/'.format(videos) + filename)
            # store loaded image
            order=int(filename[0:8]) # Dizi sırasını normalize ettik.
            loaded_images[order]=img_data


        frames_gt = []
        for line in open('VOT2019_LT/gt/{}_groundtruth.txt'.format(videos),"r").readlines():
            X = line.strip() 
            frames_gt.append(X.split(","))


        grey_eight_imgs_gt = {}
        for i in range(len(frames_gt)):
            grey_eight_imgs_gt[i]= cut_and_gray(loaded_images[i+1],frames_gt[i])


        boat_masks = {}
        frame_number = {}
        for line in open('VOT2019_LT/Mask/{}_mask'.format("ballet"),"r").readlines():
            M = line.strip()
            splitted = M.split("\t")
            try:
                boat_masks[int(splitted[0])].append(splitted[1:5])
            except KeyError:
                boat_masks[int(splitted[0])] = [splitted[1:5]]


        grey_mask = {}
        #order is a number that how many image i have
        for mask in range(1,(order+1)):
            concat = []
            if mask in boat_masks:
                for val in boat_masks[mask]:
                    eight_pics = cut_and_gray(loaded_images[mask], val)
                    for a in range(0,8):
                        concat.append(local_binary_pattern(eight_pics[a], n_points, radius, "default"))

                    if mask in grey_mask:
                        grey_mask[mask].append(concat)
                    else:
                        grey_mask[mask] = [concat]
            else:
                grey_mask[mask] = -2


        stack_mask = stack(grey_mask)
        mask_hists = hist_generator(stack_mask)
        concat_gt_d = concat_lbp(grey_eight_imgs_gt, 'default')
        stack_gt = stack(concat_gt_d)
        gt_hists = hist_generator(stack_gt)

        distance_gt_mask = dist_collector(gt_hists, mask_hists)
        distances_gt_gt = dist_collector2(gt_hists)

        with open('distances/{}_distances.txt'.format(videos), 'w') as f:
            for fr in range(len(distance_gt_mask)):
                string = distance_gt_mask[fr][0]
                for dists in distance_gt_mask[fr][1:]:
                    string = str(string) + "," + str(dists)
                f.write(str(int(fr)+1)+","+str(string)+"\n")

'Bicycle video frame started!'
